In [1]:
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../../')
from EIR_overlay_analysis import *
import pyproj

ModuleNotFoundError: No module named 'fiona'

In [32]:
layer = 'doc_farmland_mapping_and_monitoring_program_2018'

In [33]:
data_inventory = load_data_inventory()
fname, colname = load_inventory_row(layer, data_inventory)

In [19]:
df = load_overlay_file(fname, layer)

RuntimeError: b'no arguments in initialization list'

In [34]:
df = gpd.read_file(fname,
                     layer=layer,
                     driver='FileGDB')


In [36]:
#df.crs = {'init' :'epsg:3857'}
df = df.to_crs({'EPSG:32610'})

AttributeError: 'set' object has no attribute 'items'

In [ ]:
df['polygon_ty'].unique()

In [7]:
df['polygon_ty'].unique()

array(['G', 'L', 'nv', 'P', 'S', 'sAC', 'U', 'V', 'D', 'Cl', 'R', 'X',
       'W', 'LP', 'Z'], dtype=object)

In [11]:
fname, colname

('/Users\\ywang\\Box\\Plan Bay Area 2050 - EIR Analysis and Data\\pba2050_eirFeatureData1.gdb',
 'fmmp_farmland_acres')

In [6]:
df.head()

,county_nam,county,polygon_ty,upd_year,polygon_ac,metalink,popupinfo,Shape_Length,Shape_Area,geometry
0,fre,Fresno,G,2018,504164.833284,http://maps.conservation.ca.gov/dlrp/metadata/...,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",2.082572,1.256950e-04,"MULTIPOLYGON (((741760.178 3976995.905, 741968..."
1,fre,Fresno,L,2018,304.481573,http://maps.conservation.ca.gov/dlrp/metadata/...,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",0.006637,4.252086e-07,"MULTIPOLYGON (((698034.272 4080959.722, 697880..."
2,fre,Fresno,L,2018,174.217310,http://maps.conservation.ca.gov/dlrp/metadata/...,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",0.003049,2.786200e-08,"MULTIPOLYGON (((698401.348 4081361.327, 698401..."
3,fre,Fresno,nv,2018,753.388844,http://maps.conservation.ca.gov/dlrp/metadata/...,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",0.000540,9.140294e-10,"MULTIPOLYGON (((718599.554 4102610.412, 718581..."
4,fre,Fresno,P,2018,54826.894668,http://maps.conservation.ca.gov/dlrp/metadata/...,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",0.043430,1.743283e-06,"MULTIPOLYGON (((698646.570 4081614.508, 698641..."


In [4]:
data_inventory.head()

,Database Schema,layer,layer_name,Source\n(Updated to source of PBA 2050 data set),PBA 2040 Version,Available Version,"Have Latest Version in FGDB\n(Yes, No, N/A)","Metadata Status [Needs, Partial, or Complete]",Chapter,Lead,PBA 2040 Map Figure,PBA50 Map Figure,Class,PBA50 Analysis Criteria,PBA50 Output Fieldname,FGDB Location,In Enterprise Database,On Socrata
0,Basemap,Airports,cdot_mtc_airport_boundaries_2015,California Department of Transportation (Caltr...,2011,2015,Yes,Needs,3.9 HAZ,Consultant,2.13-3,3.9-3,Public Airport;\nPublic Use Airport,Within 2-mile proximity,num_airports,2.0,Y,Y
1,Basemap,California Air Districts - region,carb_airDistricts_2004,California Air Resources Board,2004,2004,Yes,Needs,3.4 AQ,Harold,2.2-1,3.4-1,NaN,NaN,NaN,NaN,Y,Y
2,Basemap,City Boundary Poly,tomtom_cityBoundary_2019,TomTom North America,2016,2019,Yes,Partial,NaN,NaN,all,all,NaN,NaN,NaN,NaN,Y,Y
3,Basemap,Counties Shoreline Clipped,counties_shorelineClipped_2019,TomTom North America; Metropolitan Transportat...,2016,2019,Yes,Needs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y
4,Basemap,County Boundary Lines,tomtom_countyBoundaryLine_2019,TomTom North America,2016,2019 (TomTom),Yes,Partial,NaN,NaN,all,all,NaN,NaN,NaN,NaN,Y,Y


In [ ]:
data_inventory = load_data_inventory()
acreage_layers = get_acreage_layers(data_inventory)
acreage_layers

In [ ]:
habitat_layers = get_habitat_layers(data_inventory)
is_ready = '~`GDB Location`.isnull()'
not_habitat = '~layer_name.isin(@habitat_layers)'
is_acreage = "`PBA50 Analysis Criteria` == 'Acres of Overlap'"

q_list = [is_ready,
          not_habitat,
          is_acreage]

q = ' & '.join(['({})'.format(query) for query in q_list])

acreage_layers = (data_inventory[data_inventory.eval(q, engine='python')]
                  ['layer_name'].unique())

In [ ]:
data_inventory[data_inventory.eval(q, engine='python')]

### List inventory layers

The column **GDB Location** was created through manual matching of layer names to the file contents below. Some layer names were updated in the *FeatureClass* column

In [ ]:
gdbname1 = os.path.join(dirname, 'pba2050_eirFeatureData1.gdb')
layers = fiona.listlayers(gdbname1)
layers

In [ ]:
write_lines(layers, 'pba2050_eirFeatureData1_layers.txt')

In [ ]:
gdbname2 = os.path.join(dirname, 'pba2050_eirFeatureData2.gdb')
layers = fiona.listlayers(gdbname2)
layers

In [ ]:
write_lines(layers, 'pba2050_eirFeatureData2_layers.txt')

### Data inventory

https://mtcdrive.app.box.com/file/701734740559?s=43jc8eapmgh4jynd0hcwmkyu9cewfkpk

In [ ]:
data_inventory = pd.read_excel(os.path.join(dirname,
                                            'PBA 2050 DEIR Data Inventory.xlsx')
                              )
data_inventory = data_inventory.rename(columns={'Layer Name': 'layer'})
data_inventory.columns.values[2] = 'layer_name'
data_inventory.head()

In [ ]:
def get_habitat_layers(data_inventory):
    q = "layer.str.startswith('Critical Habitat', na=False)"
    habitat_layers = (data_inventory[data_inventory.eval(q, engine='python')]
                      ['layer_name'].unique())
    write_lines(habitat_layers, 'EIR_habitat_layers.txt')
    return habitat_layers


def get_acreage_layers(data_inventory):
    """
    Returns a list of layer names for which we will do an
    overlay acreage analysis
    """
    is_ready = '~`GDB Location`.isnull()'
    not_habitat = '~layer.isin(@habitat_layers)'
    is_acreage = "`PBA50 Analysis Criteria` == 'Acres of Overlap'"

    q_list = [is_ready,
              not_habitat,
              is_acreage]

    q = ' & '.join(['({})'.format(query) for query in q_list])

    acreage_layers = (data_inventory[data_inventory.eval(q, engine='python')]
                      ['layer_name'].unique())
    return acreage_layers

In [ ]:
layer = acreage_layers[-1]
layer

In [ ]:
def combine_williamson_dfs(fname, layer):
    if 'williamson' not in layer:
        return
    williamson_layers = layer.split()
    df_list = []
    for l in williamson_layers:
        df_list.append(load_overlay_file(fname, l))
    williamson_df = pd.concat(df_list,
                              ignore_index=True,
                              sort=False)
#     print(len(williamson_df))  # 12043
    return williamson_df

In [ ]:
if 'williamson' in layer:
    williamson_layers = layer.split()
williamson_layers

In [ ]:
fname, output_colname = load_inventory_row(layer)
fname, output_colname

In [ ]:
l = williamson_layers[0]

In [ ]:
df_list = []
for l in williamson_layers:
    df_list.append(load_file(fname, l))
williamson_df = pd.concat(df_list,
                          ignore_index=True,
                          sort=False)
len(williamson_df)  # 12043

In [ ]:
layer = 'cdfw_essentialConnectivityAreas_2010'

In [ ]:
q = 'layer_name == @layer'
row = data_inventory[data_inventory.eval(q, engine='python')]
fname = row['GDB Location'].iloc[0]
fname

In [ ]:
row

In [ ]:
colname = row['PBA50 Output Fieldname'].iloc[0]
colname

In [ ]:
def cdfw_essential_conn_acres(final_d):
    """
    Essential Connectivity Area: Acres of Overlap
    """
    # fname = gdbname1
    layer = 'cdfw_essentialConnectivityAreas_2010'
    colname = 'cdfw_essential_conn_acres'

    final = get_acre_sum(fname, layer, colname)
    final_d[colname] = dict(zip(final['countyname'], final[colname]))

In [ ]:
def load_inventory_row(layer):
    q = 'layer_name == @layer'
    row = data_inventory[data_inventory.eval(q, engine='python')]
    # FileGDB location
    try:
        gdb_loc = int(row['GDB Location'].iloc[0])
    except:
        print('Update the data inventory to specify the FileGDB location')
        return
    fname = os.path.join(dirname, 'pba2050_eirFeatureData{}.gdb'.format(gdb_loc))
    # output field name
    output_colname = row['PBA50 Output Fieldname'].iloc[0]
    return fname, output_colname

### Load project footprint

In [ ]:
fname = os.path.join(dirname,
                     'Data Collection',
                     'Project Footprint',
                     'SLR Footprint',
                     'SLR-Footprint_analysis.geojson')
slr = gpd.read_file(fname, driver='GeoJSON')
print(slr.crs)  # epsg:32610
slr.head()

In [ ]:
slr.plot()

In [ ]:
# split project footprint by county
counties, water = get_counties_water()
slr = gpd.overlay(slr.drop('Acres', axis=1),
                  counties,
                  how='intersection')

slr['land_acres'] = slr['geometry'].area * acres_per_m

### Define functions + initialize results

In [ ]:
final_d = {}

In [ ]:
# get county base df for mapping dicts
county_df = slr[['countyname']].drop_duplicates()

In [ ]:
def load_file(fname, layer):
    """
    Given a desired overlay filename and layer, returns the
    overlay as a GeoDataFrame projected to EPSG:32610
    """
    df = gpd.read_file(fname,
                         layer=layer,
                         driver='FileGDB')
    df = df.to_crs('EPSG:32610')
    return df

In [ ]:
def get_acre_sum(fname, layer, colname):
    """
    Given a desired overlay filename, layer, and output column name,
    returns a dataframe of the county and total acreage
    """
    df = load_file(fname, layer)

    joined = gpd.overlay(slr,
                         df,
                         how='intersection')

    joined['acres'] = joined['geometry'].area * acres_per_m

    groupby_cols = ['countyname']
    agg_d = {'acres': 'sum'}
    rename_d = {'acres': colname}

    final = (joined
             .groupby(groupby_cols, as_index=False)
             .agg(agg_d)
             .rename(columns=rename_d)
            )
    return final

## 1. Airports

Within 2-mile proximity

In [ ]:
fname = gdbname2
layer = 'cdot_mtc_airport_boundaries_2015'
colname = 'num_airports'

In [ ]:
df = load_file(fname, layer)

m_in_2mi = 3218.69

slr_buff = slr.copy()
slr_buff['geom_buff'] = slr_buff['geometry'].buffer(m_in_2mi)
slr_buff = gpd.sjoin(slr_buff.set_geometry('geom_buff'),
                     df,
                     how='left')

groupby_cols = ['countyname']
agg_d = {'index_right': 'nunique'}
rename_d = {'index_right': colname}

final = (slr_buff
         .groupby(groupby_cols, as_index=False)
         .agg(agg_d)
         .rename(columns=rename_d)
        )

final_d[colname] = dict(zip(final['countyname'], final[colname]))

## 2. CASGEM Groundwater Basin Prioritization

Acres of Overlap

In [ ]:
fname = gdbname1
layer = 'dwr_groundwaterBasinPrioritization_2019'
colname = 'casgem_gwater_acres'

In [ ]:
final = get_acre_sum(fname, layer, colname)
final_d[colname] = dict(zip(final['countyname'], final[colname]))

## 3. Critical Habitat

Number of species intersected

In [ ]:
colname = 'num_species'

habitat_d = {}
fname = gdbname1

In [ ]:
q = "layer.str.startswith('Critical Habitat', na=False)"
habitat_layers = (data_inventory[data_inventory.eval(q, engine='python')]
                  ['layer_name'].unique())

for idx, layer in enumerate(habitat_layers):
    cname = 'habitat_{}'.format(idx)
    try:
        final = get_acre_sum(fname, layer, cname)
        habitat_d[layer] = dict(zip(final['countyname'], final[cname]))
    except:
        habitat_d[layer] = {}

# map habitat areas to county df
final_df = county_df.copy()
for idx, k in enumerate(habitat_d):
    final_df['habitat_{}'.format(idx)] = final_df['countyname'].map(habitat_d[k])

# count number of species intersected per county
habitat_cols = [c for c in final_df if c.startswith('habitat')]
for c in habitat_cols:
    final_df[c] = (~final_df[c].isnull()) * 1

final_df[colname] = final_df[habitat_cols].sum(axis=1)

final = final_df.drop(habitat_cols, axis=1)

# add to final dict
final_d[colname] = dict(zip(final['countyname'], final[colname]))

## Assemble final dataframe from dictionaries

In [ ]:
final_df = county_df.copy()
for k in final_d:
    final_df[k] = final_df['countyname'].map(final_d[k])
final_df

In [ ]:
final_df.to_csv('SLR_overlay_analysis.csv', index=False)